In [2]:
import pandas as pd
import numpy as np
!pip install pandas-ta optuna
import pandas_ta as ta
from tensorflow.keras.layers import LSTM,Dense,Dropout,BatchNormalization,Bidirectional,Input,Conv1D,Flatten,Conv1D
from tensorflow.keras.activations import relu,sigmoid,tanh,linear
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
# from keras_tuner.tuners import BayesianOptimization
# from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Dropout, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
# from keras_tuner.tuners import BayesianOptimization
from tensorflow.keras.callbacks import EarlyStopping
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split

import optuna



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.3 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=aee6592e3c167040a2d5e849542c2435c3c16d2bcabf683416de1d1ca65a9ffa
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta


In [3]:
df = pd.read_csv('/content/drive/MyDrive/DATA/SPX_Data/sap500_daily.csv')

In [4]:
df = df.drop(columns='Volume',axis=1)

In [5]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close'], dtype='object')

In [6]:
# import pandas as pd
# df['Date'] = pd.to_datetime(df['Date'])
# df['week'] = df['time'].dt.day_name()
df.set_index('Date',inplace=True)
# print(df['week'].unique)

In [7]:
df['next_hour_Close'] = df['Close'].shift(-1)
df['ema5'] = ta.ema(df['Close'] , length=5)
df['ema10'] = ta.ema(df['Close'] , length=10)
# delta = df['Close'].diff()
df['upper_shadow'] = df['High'] - np.maximum(df['Open'], df['Close'])
df['Lower_shadow'] = np.minimum(df['Open'], df['Close']) - df['Low']
df['body'] = abs(df['Close'] - df['Open'])
df['candle_range'] = df['High'] - df['Low']
df['atr'] = df['High'] - df['Low']
df['atr'] = df['atr'].rolling(14).mean()

# gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
# loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
# rs = gain / loss
# df['RSI'] = 100 - (100 / (1 + rs))
# df['ema20'] = ta.ema(df['Close'] , length=20)
# df['ema50'] = ta.ema(df['Close'] , length=50)
# df['ema200'] = ta.ema(df['Close'] , length=200)
df['rsi'] = ta.rsi(df['Close'] , length=14)
macd = ta.macd(df['Close'], fast=12, sLow=26, signal=9)
df = pd.concat([df,macd],axis=1)
df['pivot'] = (df['High'] + df['Low'] + df['Close']) / 3
df['support_1'] = (2 * df['pivot']) - df['High']
df['support_2'] = df['pivot'] - (df['High'] - df['Low'])
df['resistance_1'] = (2 * df['pivot']) - df['Low']
df['resistance_2'] = df['pivot'] + (df['High'] - df['Low'])
bollinger = ta.bbands(df['Close'],length=20,std=2)
df = pd.concat([df,bollinger],axis=1)
stochastic = ta.stoch(df['High'], df['Low'], df['Close'], k=14, d=3, smooth_k=3)
df = pd.concat([df, stochastic], axis=1)
adx = ta.adx(df['High'], df['Low'], df['Close'], length=14)
df = pd.concat([df, adx], axis=1)
def fibonacci_retracement(df):
    High = df['High'].max()
    Low = df['Low'].min()

    diff = High - Low

    df['fibonaccilevel2'] = High - 0.382 * diff
    df['fibonaccilevel3'] = High - 0.5 * diff
    df['fibonaccilevel4'] = High - 0.618 * diff
    df['fibonaccilevel5'] = High - 0.764 * diff

    return df

# fibonacci_retracement(df)

def stochastic_oscillator(df, period=14, ma_period=3):
    df['L14'] = df['Low'].rolling(window=period).min()
    df['H14'] = df['High'].rolling(window=period).max()
    df['%K'] = (df['Close'] - df['L14']) * 100 / (df['H14'] - df['L14'])
    df['%D'] = df['%K'].rolling(window=ma_period).mean()
    return df
stochastic_oscillator(df)

# Additional pivot-based levels
df['support_3'] = df['Low'] - 2 * (df['High'] - df['pivot'])
df['resistance_3'] = df['High'] + 2 * (df['pivot'] - df['Low'])
# Fibonacci levels (common ratios)
fib_range = df['High'] - df['Low']
df['fib_236'] = df['High'] - fib_range * 0.236
df['fib_382'] = df['Close'] + fib_range * 0.382  # Common retracement
df['fib_618'] = df['Close'] - fib_range * 0.618  # Golden ratio
# Woodie's variation (uses 2*Close in calculation)
df['woodie_pivot'] = (df['High'] + df['Low'] + 2 * df['Close']) / 4
df['woodie_s1'] = 2 * df['woodie_pivot'] - df['High']
df['woodie_r1'] = 2 * df['woodie_pivot'] - df['Low']
# Camarilla system (uses different multipliers)
df['camarilla_s1'] = df['Close'] - (df['High'] - df['Low']) * 1.1 / 4
df['camarilla_r1'] = df['Close'] + (df['High'] - df['Low']) * 1.1 / 4
# Midpoints between key levels
df['mid_support'] = (df['support_1'] + df['support_2']) / 2
df['mid_resistance'] = (df['resistance_1'] + df['resistance_2']) / 2
# Alternative pivot calculation (includes Open price)
df['alt_pivot'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
# Moving average of pivot points
df['pivot_ma7'] = df['pivot'].rolling(window=7).mean()
df['pivot_ma21'] = df['pivot'].rolling(window=21).mean()
# Price range calculations
df['daily_range'] = df['High'] - df['Low']
df['range_pct'] = (df['daily_range'] / df['Close']) * 100

df['HA_Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
df['HA_Open'] = (df['Open'].shift(1) + df['Close'].shift(1)) / 2
df['HA_High'] = df[['High', 'HA_Open', 'HA_Close']].max(axis=1)
df['HA_Low'] = df[['Low', 'HA_Open', 'HA_Close']].min(axis=1)
df['HA_Open'].iloc[0] = (df['Open'].iloc[0] + df['Close'].iloc[0]) / 2


<ipython-input-7-ac4c52fa0d3f>:87: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['HA_Open'].iloc[0] = (df['Open'].iloc[0] + df['Close'].iloc[0]) / 2


In [8]:
df.isnull().sum()


,0
Open,0
High,0
Low,0
Close,0
next_hour_Close,1
ema5,4
ema10,9
upper_shadow,0
Lower_shadow,0
body,0


In [9]:
df.dropna(inplace=True)

In [10]:
count_indices = df.index.size

print(count_indices)


24374


In [11]:
features = [
#  'Open',
#  'High',
#  'Low',
#  'Close',
#  'Volume',
#  'ema5',
#  'ema10',
#  'upper_shadow',
#  'Lower_shadow',
#  'body',
#  'candle_range',
#  'atr',
#  'rsi',
#  'MACD_12_26_9',
#  'MACDh_12_26_9',
#  'MACDs_12_26_9',
#  'pivot',
#  'support_1',
#  'support_2',
#  'resistance_1',
#  'resistance_2',
#  'BBL_20_2.0',
  'BBM_20_2.0',
  'BBU_20_2.0',
#  'BBB_20_2.0',
#  'BBP_20_2.0',
#  'STOCHk_14_3_3',
#  'STOCHd_14_3_3',
#  'ADX_14',
#  'DMP_14',
#  'DMN_14',
#  'L14',
#  'H14',
#  '%K',
#  '%D',
#  'support_3',
  'resistance_3',
#  'fib_236',
#  'fib_382',
#  'fib_618',
#  'woodie_pivot',
#  'woodie_s1',
#  'woodie_r1',
  'camarilla_s1',
#  'camarilla_r1',
#  'mid_support',
#  'mid_resistance',
#  'alt_pivot',
#  'pivot_ma7',
  'pivot_ma21',
#  'daily_range',
#  'range_pct',
#  'HA_Close',
#  'HA_Open',
#  'HA_High',
#  'HA_Low'
]
target = ['next_hour_Close']

# close              0.999833
# camarilla_s1       0.999827
# camarilla_r1       0.999825
# woodie_pivot       0.999824
# fib_382            0.999819
# pivot              0.999818
# woodie_s1          0.999813
# support_1          0.999810
# woodie_r1          0.999809
# resistance_1       0.999804
# alt_pivot          0.999803
# fib_618            0.999802
# low                0.999782
# mid_support        0.999779
# high               0.999773
# mid_resistance     0.999770
# support_2          0.999737
# resistance_2       0.999722
# open               0.999720
# ema5               0.999698
# support_3          0.999639
# resistance_3       0.999621
# ema10              0.999494
# pivot_ma7          0.999487
# ema20              0.999074
# L14                0.998902
# BBM_20_2.0         0.998797
# H14                0.998785
# pivot_ma21         0.998697
# BBL_20_2.0         0.998270
# BBU_20_2.0         0.998088
# ema50              0.997838
# ema200             0.992243

In [12]:
X = df[features].values
y = df[target].values.reshape(-1, 1)

In [13]:
x_scaler = StandardScaler()
y_scaler = StandardScaler()


X_scaled = x_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

In [14]:
Sequence_len = 40

def creat_sequences(X,y,sequence_len):
  X_seq = []
  y_seq = []
  for i in range(len(X)-sequence_len):
    X_seq.append(X[i:i+sequence_len])
    y_seq.append(y[i+sequence_len])
  return np.array(X_seq),np.array(y_seq)

In [15]:
X_final,y_final = creat_sequences(X_scaled,y_scaled,Sequence_len)

In [16]:
print(X_final.shape)
print(y_final.shape)

(24334, 40, 5)
(24334, 1)


In [17]:
split_index = int(len(X_final)*0.90)
x_train = X_final[:split_index]
y_train = y_final[:split_index]
x_test = X_final[split_index:]
y_test = y_final[split_index:]

In [18]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(21900, 40, 5)
(21900, 1)
(2434, 40, 5)
(2434, 1)


In [19]:
def objective(trial):
    # هایپرپارامترهای پایه
    n_lstm_layers = trial.suggest_int("n_lstm_layers", 1, 3)
    n_dense_layers = trial.suggest_int("n_dense_layers", 1, 3)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # تعریف پارامترهای مجزا برای هر لایه LSTM
    lstm_units_list = [
        trial.suggest_int(f"lstm_units_{i}", 32, 512, step=32)
        for i in range(3)  # حداکثر 3 لایه
    ]
    lstm_dropout = [
        trial.suggest_float(f"lstm_dropout_{i}", 0.0, 0.5, step=0.1)
        for i in range(3)
    ]
    lstm_l2 = [
        trial.suggest_float(f"lstm_l2_{i}", 1e-6, 1e-2, log=True)
        for i in range(3)
    ]

    # تعریف پارامترهای مجزا برای هر لایه Dense
    dense_units_list = [
        trial.suggest_int(f"dense_units_{i}", 16, 256, step=16)
        for i in range(3)  # حداکثر 3 لایه
    ]
    dense_dropout = [
        trial.suggest_float(f"dense_dropout_{i}", 0.0, 0.5, step=0.1)
        for i in range(3)
    ]
    dense_l2 = [
        trial.suggest_float(f"dense_l2_{i}", 1e-6, 1e-2, log=True)
        for i in range(3)
    ]

    # ساخت مدل
    model = Sequential()

    # اولین لایه LSTM
    model.add(Bidirectional(
        LSTM(
            lstm_units_list[0],
            return_sequences=(n_lstm_layers > 1),
            kernel_regularizer=l2(lstm_l2[0])  # L2 مجزا برای لایه اول
        ),
        input_shape=(Sequence_len, x_train.shape[2])
    ))
    model.add(Dropout(lstm_dropout[0]))  # Dropout مجزا برای لایه اول

    # سایر لایههای LSTM
    for i in range(1, n_lstm_layers):
        return_sequences = (i < n_lstm_layers - 1)
        model.add(Bidirectional(
            LSTM(
                lstm_units_list[i],
                return_sequences=return_sequences,
                kernel_regularizer=l2(lstm_l2[i])  # L2 مجزا
            )
        ))
        model.add(Dropout(lstm_dropout[i]))  # Dropout مجزا

    # لایههای Dense
    for i in range(n_dense_layers):
        model.add(Dense(
            dense_units_list[i],
            activation='relu',
            kernel_regularizer=l2(dense_l2[i])  # L2 مجزا برای Dense
        ))
        model.add(Dropout(dense_dropout[i]))  # Dropout مجزا

    # لایه خروجی
    model.add(Dense(1, activation='linear'))

    # کامپایل مدل
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )
    Es = EarlyStopping(monitor='val_mae',patience=10)


    # آموزش مدل
    history = model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        batch_size=batch_size,
        epochs=30,
        verbose=1,
        callbacks=[Es]
    )

    return min(history.history["val_loss"])

In [ ]:
study = optuna.create_study(direction="minimize")  # هدف کاهش loss است
study.optimize(objective, n_trials=50)  # تعداد اجراهای مختلف

[I 2025-03-12 13:15:59,260] A new study created in memory with name: no-name-cd096e0a-b908-4f4a-b9da-fee49c15ba01
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1369/1369 ━━━━━━━━━━━━━━━━━━━━ 72s 40ms/step - loss: 0.1507 - mae: 0.1057 - val_loss: 2.2693 - val_mae: 1.2499
Epoch 2/30
1369/1369 ━━━━━━━━━━━━━━━━━━━━ 78s 40ms/step - loss: 0.0121 - mae: 0.0498 - val_loss: 2.9924 - val_mae: 1.4484
Epoch 3/30
1369/1369 ━━━━━━━━━━━━━━━━━━━━ 82s 40ms/step - loss: 0.0075 - mae: 0.0466 - val_loss: 3.0872 - val_mae: 1.4604
Epoch 4/30
1369/1369 ━━━━━━━━━━━━━━━━━━━━ 82s 40ms/step - loss: 0.0058 - mae: 0.0427 - val_loss: 3.1279 - val_mae: 1.4605
Epoch 5/30
1369/1369 ━━━━━━━━━━━━━━━━━━━━ 82s 40ms/step - loss: 0.0055 - mae: 0.0420 - val_loss: 2.9511 - val_mae: 1.3973
Epoch 6/30
1369/1369 ━━━━━━━━━━━━━━━━━━━━ 54s 39ms/step - loss: 0.0055 - mae: 0.0427 - val_loss: 3.3970 - val_mae: 1.5562
Epoch 7/30
1369/1369 ━━━━━━━━━━━━━━━━━━━━ 82s 39ms/step - loss: 0.0051 - mae: 0.0419 - val_loss: 2.7680 - val_mae: 1.3417
Epoch 8/30
1369/1369 ━━━━━━━━━━━━━━━━━━━━ 54s 39ms/step - loss: 0.0050 - mae: 0.0423 - val_loss: 3.1513 - val_mae: 1.4735
Epoch 9/30
1369/1369 ━━━

[I 2025-03-12 13:29:54,418] Trial 0 finished with value: 2.2693355083465576 and parameters: {'n_lstm_layers': 3, 'n_dense_layers': 2, 'learning_rate': 0.0004759672352335635, 'batch_size': 16, 'lstm_units_0': 448, 'lstm_units_1': 352, 'lstm_units_2': 480, 'lstm_dropout_0': 0.4, 'lstm_dropout_1': 0.5, 'lstm_dropout_2': 0.30000000000000004, 'lstm_l2_0': 0.0006454512625975572, 'lstm_l2_1': 1.1775731609451228e-06, 'lstm_l2_2': 9.726804763298005e-06, 'dense_units_0': 16, 'dense_units_1': 160, 'dense_units_2': 48, 'dense_dropout_0': 0.2, 'dense_dropout_1': 0.2, 'dense_dropout_2': 0.4, 'dense_l2_0': 1.11323304412953e-05, 'dense_l2_1': 0.008916240895994987, 'dense_l2_2': 2.138324819853209e-05}. Best is trial 0 with value: 2.2693355083465576.


Epoch 1/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - loss: 0.0792 - mae: 0.1447 - val_loss: 1.1951 - val_mae: 0.8117
Epoch 2/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - loss: 0.0332 - mae: 0.0580 - val_loss: 0.8424 - val_mae: 0.6565
Epoch 3/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - loss: 0.0280 - mae: 0.0517 - val_loss: 0.6299 - val_mae: 0.5396
Epoch 4/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - loss: 0.0243 - mae: 0.0490 - val_loss: 0.7260 - val_mae: 0.6244
Epoch 5/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - loss: 0.0211 - mae: 0.0464 - val_loss: 0.3968 - val_mae: 0.4112
Epoch 6/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - loss: 0.0186 - mae: 0.0457 - val_loss: 0.3001 - val_mae: 0.3374
Epoch 7/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - loss: 0.0165 - mae: 0.0436 - val_loss: 0.2737 - val_mae: 0.3193
Epoch 8/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - loss: 0.0149 - mae: 0.0426 - val_loss: 0.1641 - val_mae: 0.2329
Epoch 9/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 7s

[I 2025-03-12 13:34:11,576] Trial 1 finished with value: 0.04047619178891182 and parameters: {'n_lstm_layers': 2, 'n_dense_layers': 1, 'learning_rate': 3.0796675844787444e-05, 'batch_size': 64, 'lstm_units_0': 160, 'lstm_units_1': 224, 'lstm_units_2': 480, 'lstm_dropout_0': 0.4, 'lstm_dropout_1': 0.30000000000000004, 'lstm_dropout_2': 0.0, 'lstm_l2_0': 6.36241848133251e-06, 'lstm_l2_1': 3.3302476848799786e-05, 'lstm_l2_2': 8.678630059444509e-05, 'dense_units_0': 144, 'dense_units_1': 240, 'dense_units_2': 192, 'dense_dropout_0': 0.30000000000000004, 'dense_dropout_1': 0.2, 'dense_dropout_2': 0.5, 'dense_l2_0': 3.192627615360871e-06, 'dense_l2_1': 3.349285187875852e-05, 'dense_l2_2': 0.00018448313776238368}. Best is trial 1 with value: 0.04047619178891182.


...

Epoch 1/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - loss: 0.1197 - mae: 0.1382 - val_loss: 1.4419 - val_mae: 0.8811
Epoch 2/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - loss: 0.0588 - mae: 0.0666 - val_loss: 1.0584 - val_mae: 0.7159
Epoch 3/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - loss: 0.0408 - mae: 0.0597 - val_loss: 0.7372 - val_mae: 0.5637
Epoch 4/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0307 - mae: 0.0564 - val_loss: 0.7978 - val_mae: 0.6254
Epoch 5/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - loss: 0.0252 - mae: 0.0546 - val_loss: 0.5753 - val_mae: 0.5013
Epoch 6/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0215 - mae: 0.0527 - val_loss: 0.5824 - val_mae: 0.5187
Epoch 7/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0193 - mae: 0.0525 - val_loss: 0.5054 - val_mae: 0.4856
Epoch 8/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - loss: 0.0175 - mae: 0.0510 - val_loss: 0.3712 - val_mae: 0.3905
Epoch 9/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms

[I 2025-03-12 14:29:42,216] Trial 12 finished with value: 0.08288165926933289 and parameters: {'n_lstm_layers': 2, 'n_dense_layers': 1, 'learning_rate': 7.20727071756022e-05, 'batch_size': 64, 'lstm_units_0': 64, 'lstm_units_1': 160, 'lstm_units_2': 288, 'lstm_dropout_0': 0.4, 'lstm_dropout_1': 0.30000000000000004, 'lstm_dropout_2': 0.2, 'lstm_l2_0': 4.5983359576982715e-05, 'lstm_l2_1': 0.00019215221642461967, 'lstm_l2_2': 0.0007148341200079594, 'dense_units_0': 112, 'dense_units_1': 96, 'dense_units_2': 192, 'dense_dropout_0': 0.4, 'dense_dropout_1': 0.5, 'dense_dropout_2': 0.4, 'dense_l2_0': 1.2009183874504733e-05, 'dense_l2_1': 0.00020704334861117797, 'dense_l2_2': 0.0002959093942826147}. Best is trial 4 with value: 0.039175845682621.


Epoch 1/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - loss: 0.1022 - mae: 0.2157 - val_loss: 0.7321 - val_mae: 0.6148
Epoch 2/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.0199 - mae: 0.0553 - val_loss: 0.5153 - val_mae: 0.4899
Epoch 3/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0187 - mae: 0.0498 - val_loss: 0.4490 - val_mae: 0.4554
Epoch 4/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 0.0181 - mae: 0.0467 - val_loss: 0.3689 - val_mae: 0.4059
Epoch 5/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 0.0175 - mae: 0.0438 - val_loss: 0.2957 - val_mae: 0.3541
Epoch 6/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 0.0167 - mae: 0.0408 - val_loss: 0.2357 - val_mae: 0.3077
Epoch 7/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.0163 - mae: 0.0391 - val_loss: 0.2236 - val_mae: 0.3062
Epoch 8/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 0.0157 - mae: 0.0373 - val_loss: 0.1873 - val_mae: 0.2758
Epoch 9/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 10s

[I 2025-03-12 14:34:36,213] Trial 13 finished with value: 0.029697731137275696 and parameters: {'n_lstm_layers': 1, 'n_dense_layers': 2, 'learning_rate': 1.0489716937127874e-05, 'batch_size': 32, 'lstm_units_0': 256, 'lstm_units_1': 256, 'lstm_units_2': 416, 'lstm_dropout_0': 0.4, 'lstm_dropout_1': 0.4, 'lstm_dropout_2': 0.4, 'lstm_l2_0': 1.1662429082781385e-06, 'lstm_l2_1': 3.8154019870560715e-06, 'lstm_l2_2': 0.00011207587009691215, 'dense_units_0': 208, 'dense_units_1': 208, 'dense_units_2': 208, 'dense_dropout_0': 0.1, 'dense_dropout_1': 0.1, 'dense_dropout_2': 0.5, 'dense_l2_0': 4.364174041564781e-05, 'dense_l2_1': 8.035711863680585e-06, 'dense_l2_2': 0.0012036498703678624}. Best is trial 13 with value: 0.029697731137275696.


Epoch 1/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 0.1110 - mae: 0.2208 - val_loss: 0.7393 - val_mae: 0.6153
Epoch 2/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0219 - mae: 0.0472 - val_loss: 0.5710 - val_mae: 0.5281
Epoch 3/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0212 - mae: 0.0436 - val_loss: 0.4761 - val_mae: 0.4811
Epoch 4/30
685/685 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0204 - mae: 0.0406

In [ ]:
print("Best hyperparameters: ", study.best_params)


In [ ]:
Es = EarlyStopping(monitor='val_mae',patience=10)
learning_rate = 0.001
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])
# history = model.fit(x_train, y_train, epochs=200, validation_data=(x_train, y_train),callbacks=Es)
history = model.fit(x_train, y_train, epochs=200, validation_data=(x_train, y_train),callbacks=Es)

In [ ]:
test_loss, test_mae = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test MAE: {test_mae}')

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test,y_pred)
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")

In [ ]:
plt.figure(figsize=(20, 20))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title('Actual vs. Predicted Close Price')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.legend()
plt.show()

In [ ]:
df_results = pd.DataFrame({
    'Actual': y_test.flatten(),
    'Predicted': y_pred.flatten()
}, index=df.index[-len(y_test):])

df_results.to_csv('daily result.csv')

print("Results saved to 'daily result.csv'")
from google.colab import files

files.download('daily result.csv')

In [ ]:
def evaluate_predictions(y_test, y_pred):
    # محاسبه معیارهای دقت مدل
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

    # بررسی جهت حرکت قیمت‌ها
    correct_direction = np.sum(np.sign(y_pred[1:] - y_pred[:-1]) == np.sign(y_test[1:] - y_test[:-1]))
    accuracy = correct_direction / (len(y_test) - 1) * 100

    print(f"Trend Direction Accuracy: {accuracy:.2f}%")

In [ ]:
evaluate_predictions(y_test, y_pred)
